[View in Colaboratory](https://colab.research.google.com/github/TimSim/SentimentAnalysis/blob/master/SentimentAnalysisWithLSTM_PreTrained.ipynb)

This is based on a tutorail published by O'Reilly https://www.oreilly.com/learning/perform-sentiment-analysis-with-lstms-using-tensorflow


Source code on which this is based can be found here: https://github.com/adeshpande3/LSTM-Sentiment-Analysis

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls -la "/content/drive/My Drive/Colab Notebooks"

total 1138455
-rw------- 1 root root     23020 Sep 27 22:28 ChatBotWithTFLearn.ipynb
-rw------- 1 root root 171350079 Oct  3 23:38 glove50.txt
-rw------- 1 root root 862182613 Oct  3 00:35 glove.6B.zip
-rw------- 1 root root  25000080 Oct  3 23:56 idsMatrix.npy
-rw------- 1 root root      2288 Sep 26 00:52 intents2.json
-rw------- 1 root root      2088 Sep 26 00:51 intents.json
-rw------- 1 root root      7902 Oct  3 23:52 SentimentAnalysisWithLSTM.ipynb
-rw------- 1 root root       315 Sep 27 22:37 Untitled0.ipynb
-rw------- 1 root root       579 Aug 31 22:34 Untitled1.ipynb
-rw------- 1 root root      4274 Oct  3 19:29 Untitled2.ipynb
-rw------- 1 root root  27200080 Oct  3 23:56 wordsList.npy
-rw------- 1 root root  80000080 Oct  3 23:56 wordVectors.npy


In [0]:
numDimensions = 300
maxSeqLength = 250
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000

In [0]:
import numpy as np
wordsList = np.load('/content/drive/My Drive/Colab Notebooks/wordsList.npy').tolist()
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('/content/drive/My Drive/Colab Notebooks/wordVectors.npy')

In [11]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [0]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('/content/drive/My Drive/Colab Notebooks/models'))